# e.g. 8: Compare theta cross-sections for monthly inputs from 2 forecast runs & control & WOA observations

# last edited 11 July 2018

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [26]:
print('BEGIN')

import socket
import re
import os
import inspect

hostname=socket.gethostname()

print('hostname=',hostname)

if(re.match('raijin',hostname)):
  print('this is rajin')
  rundir='/short/v14/mac599/cafepp/rundir'
  topdir=''
elif(re.match('oa-32-cdc',hostname)):
  print('this is vm32')
  rundir='/OSM/CBR/OA_DCFP/work/col414/cafepp'
  topdir='/OSM/CBR/OA_DCFP/data/CAFEPP/CMIP6'
  topdir_ncep='/OSM/CBR/OA_DCFP/data/CAFEPP/'

os.chdir(rundir)

os.chdir('/OSM/CBR/OA_DCFP/work/col414/cafepp')

print('Current Working Directory=',os.getcwd())

SWITCH_OFF=True #temporary for easy skipping of this section.
SWITCH_OFF=False

if(not SWITCH_OFF):
  
  %matplotlib inline

  __file__='jupyter_notebook' #this can be deleted when written to a python script and loaded as module.

  import netCDF4
  import os
  import shutil
  import numpy as np
  import numpy.ma as ma
  import matplotlib.pyplot as plt

  from decadal_diag import \
    fractional_year_from_num2date, \
    n_data_funcs, \
    shade_2d_simple

  %matplotlib inline

  ivar_woa='t_an';istr_woa='t00';ivar_cafe='thetao';units_woa='degC';units_cafe=units_woa
  #ivar_woa='s_an';istr_woa='s00';ivar_cafe='so';units_woa='pss';units_cafe='psu'

  if_woa='/OSM/CBR/OA_DCFP/data/CAFEPP/short/v14/mac599/woa/woa13_95A4_'+istr_woa+'_01v2.nc'
  fh_woa=netCDF4.Dataset(if_woa)

  var_woa = fh_woa.variables[ivar_woa][:]

  print('var_woa.shape=',var_woa.shape)

  woa_regrid=n_data_funcs(Diag=True,input_files=[[if_woa]],input_var_name=ivar_woa)

  woa_regrid.calculate_filedatetime_info(Diag=True,calendar='standard',units='days since 0001-01-01 00:00:00') #have to assign calendar/units as ones in file are not acceptable.

  woa_regrid.get_latlon_info(Diag=True)

  woa_regrid.get_lev_info(lev='depth')

  print('var_woa.shape=',var_woa.shape)

  if_cafe='/OSM/CBR/OA_DCFP/data/CAFEPP/CMIP6/CMIP6/CMIP/CSIRO/CAFE-1-0/piControl/r1i1p9f1/Omon/'+ivar_cafe+'/gn/v20171025/' \
   +ivar_cafe+'_Omon_piControl_CAFE-1-0_r1i1p9f1_gn_047101-048012.nc'
  fh_cafe=netCDF4.Dataset(if_cafe)
  
  cafe_regrid=n_data_funcs(Diag=True,input_files=[[if_cafe]],input_var_name=ivar_cafe)

  cafe_regrid.calculate_filedatetime_info(Diag=True,calendar='noleap')

  cafe_regrid.get_latlon_info(Diag=True,lat='latitude',lon='longitude')

  cafe_regrid.get_lev_info()

  woa_cafe_depth_mapping=[]
  for d,depth in enumerate(cafe_regrid.lev):
    woa_cafe_depth_mapping.append( np.abs(woa_regrid.lev - depth).argmin() )

  woa=np.average(np.squeeze(var_woa),axis=2)
  
  table='Omon'
  
  cafe_experiment_titles = {1:'v0_forecast', 4:'v1_forecast', 3:'v2_forecast', 9:'v1', 5:'v2', 6:'v3', 7:'aug17p'}
  
  pval=9

  cafe_calendar='noleap'

  idir_cafe='/OSM/CBR/OA_DCFP/data/CAFEPP/CMIP6/CMIP6/CMIP/CSIRO/CAFE-1-0/piControl/r1i1p'+str(pval)+'f1/'+table+'/'+ivar_cafe+'/gn/v20171025'
  ifils_cafe= \
    [ivar_cafe+'_'+table+'_piControl_CAFE-1-0_r1i1p'+str(pval)+'f1_gn_047101-048012.nc' \
    ,ivar_cafe+'_'+table+'_piControl_CAFE-1-0_r1i1p'+str(pval)+'f1_gn_048101-049012.nc' \
    ,ivar_cafe+'_'+table+'_piControl_CAFE-1-0_r1i1p'+str(pval)+'f1_gn_049101-050012.nc' \
    ]

  cafe_files=[[idir_cafe+'/'+ifils_cafe[0]]]

  print('cafe_files=',cafe_files)

  cafe_file_var=n_data_funcs(Diag=True,input_files=cafe_files,input_var_name=ivar_cafe)

  cafe_file_var.calculate_filedatetime_info(Diag=True,calendar=cafe_calendar)

  cafe_file_var.get_latlon_info(Diag=True, lat='latitude', lon='longitude')

  cafe_file_var.get_lev_info(Diag=True)

  cafe_global_monthly=cafe_file_var.calculate_quantity('dummy', Diag=True, quantity='latlon_region')
  
  #raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

  cafe_global_annual_climatology,cafe_global_annual_anomaly=cafe_file_var.monthly_clim_anom( \
    Diag=True, input=cafe_global_monthly, AnnOut=True, ZeroClim=True)
  
  print('cafe_global_annual_climatology.shape=',cafe_global_annual_climatology.shape)
  print('cafe_global_annual_anomaly.shape=',cafe_global_annual_anomaly.shape)

  cafe_file_var.regrid_curv_to_rect_setup( \
    Diag=False, topdir='/OSM/CBR/OA_DCFP/data/CAFEPP', outmask=True, interpolate='1x1o3d') #1x1o3d, 2x2o3d, 5x5o3d, 1x1o2d, 2x2o2d, 5x5o2d

  cafe_file_var.regrid_curv_to_rect_weights_mask(Diag=False)

  cafe_global_annual_anomaly_regrid=cafe_file_var.regrid_curv_to_rect( \
    Diag=False, input=cafe_global_annual_anomaly)

  print('cafe_global_annual_anomaly_regrid.shape=',cafe_global_annual_anomaly_regrid.shape)

  raise SystemExit('STOP!:'+__file__+' line number: '+str(inspect.stack()[0][2]))

  cafe_global_annual_climatology_regrid=cafe_file_var.regrid_curv_to_rect( \
    Diag=True, apply_lsmask=True, input=cafe_global_annual_climatology)
  
  print('cafe_global_annual_climatology_regrid.shape=',cafe_global_annual_climatology_regrid.shape)

#   cafe=np.average(cafe_control_global_annual_climatology_regrid,axis=2)

#   delta=cafe-woa[woa_cafe_depth_mapping,]

  cmap='RdBu_r'
  cmap='jet'

  xysize=(12,8)
  xysize=(6,4)

  #====

  #shade_2d_simple(woa,title='WOA thetao - zonal annual average', units=units_woa, reverse_yaxis=True,
  #  xyvals=((woa_regrid.lat,woa_regrid.lev)), xysize=xysize, xlim=(-90,90),ylim=(0,200))

  #shade_2d_simple(woa,title='WOA thetao - zonal annual average',units=units_woa, reverse_yaxis=True,
  #  xyvals=((woa_regrid.lat,woa_regrid.lev)), xysize=xysize, xlim=(-90,90), ylim=(0,1000))

  shade_2d_simple(woa,title='WOA thetao - zonal annual average', units=units_woa, reverse_yaxis=True,
    xyvals=((woa_regrid.lat,woa_regrid.lev)), \
    xysize=xysize, xlab='Latitude', ylab='Depth (m)', grid=True, cmap=cmap) #, xtik=range(-90, 90, 30), \
    #grid=True) #, clevs=[29,30,31,32,33,34,35,36])
    
print('END')

BEGIN
hostname= oa-32-cdc
this is vm32
Current Working Directory= /OSM/CBR/OA_DCFP/work/col414/cafepp
var_woa.shape= (1, 102, 180, 360)
n_data_funcs.init: no ensembles in this example.
self.input_files= [['/OSM/CBR/OA_DCFP/data/CAFEPP/short/v14/mac599/woa/woa13_95A4_t00_01v2.nc']]
calculate_filedatetime_info: case 1: no ensembles, one input file.
self.input_files= [['/OSM/CBR/OA_DCFP/data/CAFEPP/short/v14/mac599/woa/woa13_95A4_t00_01v2.nc']]
self.input_files[0][0]= /OSM/CBR/OA_DCFP/data/CAFEPP/short/v14/mac599/woa/woa13_95A4_t00_01v2.nc
var_woa.shape= (1, 102, 180, 360)
n_data_funcs.init: no ensembles in this example.
self.input_files= [['/OSM/CBR/OA_DCFP/data/CAFEPP/CMIP6/CMIP6/CMIP/CSIRO/CAFE-1-0/piControl/r1i1p9f1/Omon/thetao/gn/v20171025/thetao_Omon_piControl_CAFE-1-0_r1i1p9f1_gn_047101-048012.nc']]
calculate_filedatetime_info: case 1: no ensembles, one input file.
self.input_files= [['/OSM/CBR/OA_DCFP/data/CAFEPP/CMIP6/CMIP6/CMIP/CSIRO/CAFE-1-0/piControl/r1i1p9f1/Omon/thetao/gn/v2

AssertionError: regrid_curv_to_rect_weights_mask: self.nlon_regrid * self.nlat_regrid should equal self._A.shape[www]